Prepare data for blog post

Each event had a different fee scheme and so they have to be calculated separatly.

## Import all Stripe Transactions

In [1]:
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [2]:
# import all stripe data
stripe = pd.read_csv('data/unified_payments.csv')
stripe.head(2)

,id,Description,Seller Message,Created (UTC),Amount,Amount Refunded,Currency,Converted Amount,Converted Amount Refunded,Fee,Tax,Converted Currency,Status,Statement Descriptor,Customer ID,Customer Description,Customer Email,Captured,Card ID,Invoice ID,Transfer,client_user_agent (metadata),user_fee_amount (metadata),admin_fee_amount (metadata),contributor_id (metadata),campaign_id (metadata),customer_ip_address (metadata),customer_email (metadata),request_id (metadata),client_ip (metadata),admin_id (metadata),contribution_id (metadata),campaign_type (metadata),recipient_id (metadata),admin_country (metadata),contributor_country (metadata),recipient_country (metadata),campaign (metadata),donation (metadata),person (metadata),recurring (metadata),anonymous (metadata),on_behalf_of (metadata),Customer Name (metadata),Customer Email (metadata),customer_name (metadata),order_id (metadata),TicketTailor (metadata),name (metadata),email_address (metadata),checkout (metadata),product_0 (metadata),product_1 (metadata),Customer IP (metadata),wix_transaction_id (metadata),Order Item #1 (metadata),Invoice Id (metadata),product_2 (metadata),product_3 (metadata),Ko-fi Transaction Id (metadata),PageId (metadata),Creator Stripe Id (metadata),Product Name (metadata),Product ID (metadata),application_fee (metadata),event (metadata),event_url (metadata)
0,ch_1HxvimJAus1cTCJzro0Fjfsi,Ben & Winnie Donation,Payment complete.,2020-12-13 14:38,1.0,0.0,usd,1.0,0.0,0.32,0.0,usd,Paid,NaN,cus_IZ3q9UqMReMz4E,NaN,win109@gmail.com,True,pm_1HxvilJAus1cTCJzqzSbkRN4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ben & Winnie Donation,1331.0,NaN,NaN,NaN
1,ch_1HxvfYJAus1cTCJzn2Wwbr9q,Ben & Winnie Donation,Payment complete.,2020-12-13 14:35,10.0,0.0,usd,10.0,0.0,0.52,0.0,usd,Paid,NaN,cus_IZ3nS8S8NgSbZP,NaN,ben@benbogart.com,True,pm_1HxvfXJAus1cTCJzPgqMBGAR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ben & Winnie Donation,1331.0,NaN,NaN,NaN


In [3]:
#view the different event identifiers
stripe['event_url (metadata)'].unique()

array([nan, 'https://www.groupmuse.com/events/11161',
       'https://www.groupmuse.com/events/10728'], dtype=object)

In [4]:
# filter to only one event
event_1 = stripe[stripe['event_url (metadata)'] == 'https://www.groupmuse.com/events/10728']

# get only the columns we need
event_1 = event_1[['Amount', 'Fee', 'application_fee (metadata)', 'event (metadata)']]

len(event_1)

79

In [5]:
event_1.head(2)

,Amount,Fee,application_fee (metadata),event (metadata)
81,30.0,1.17,Stripe fees of 2.9% + $0.30 = $1.17,Ben & Winnie - 2hot2tango online
82,30.0,1.17,Stripe fees of 2.9% + $0.30 = $1.17,Ben & Winnie - 2hot2tango online


In [6]:
# filter to only one event
event_2 = stripe[stripe['event_url (metadata)'] == 'https://www.groupmuse.com/events/11161']

# get only the columns we need
event_2 = event_2[['Amount', 'Fee', 'application_fee (metadata)', 'event (metadata)']]

len(event_2)

64

In [7]:
event_2.head(2)

,Amount,Fee,application_fee (metadata),event (metadata)
3,23.0,3.0,The attendee paid you $0.0 and chose to tip Groupmuse $3.0 (the tip appears as an application fee),Argentine Tango Across Continents: Double-Band Live Show with Ben & Winnie (USA) and Asato-Pais (Argentina)
4,70.0,20.0,The attendee paid you $0.0 and chose to tip Groupmuse $20.0 (the tip appears as an application fee),Argentine Tango Across Continents: Double-Band Live Show with Ben & Winnie (USA) and Asato-Pais (Argentina)


## get donation amount before fees
For the first event the Amount is the donation amount

For the second event:
* If there was a tip to groupmuse, the `amount` includes the tips.  In this case the merchant fee was paid by groupmuse.

* If there was no tip then the Amount is the donation amount and the fee is the stripe fee.

In [8]:
event_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 64 entries, 3 to 66
Data columns (total 4 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Amount                      64 non-null     float64
 1   Fee                         64 non-null     float64
 2   application_fee (metadata)  64 non-null     object 
 3   event (metadata)            64 non-null     object 
dtypes: float64(2), object(2)
memory usage: 2.5+ KB


In [9]:
event_2

,Amount,Fee,application_fee (metadata),event (metadata)
3,23.0,3.00,The attendee paid you $0.0 and chose to tip Groupmuse $3.0 (the tip appears as an application fee),Argentine Tango Across Continents: Double-Band Live Show with Ben & Winnie (USA) and Asato-Pais (Argentina)
4,70.0,20.00,The attendee paid you $0.0 and chose to tip Groupmuse $20.0 (the tip appears as an application fee),Argentine Tango Across Continents: Double-Band Live Show with Ben & Winnie (USA) and Asato-Pais (Argentina)
5,50.0,1.75,Stripe fees of 2.9% + $0.30 = $1.75,Argentine Tango Across Continents: Double-Band Live Show with Ben & Winnie (USA) and Asato-Pais (Argentina)
6,53.0,3.00,The attendee paid you $0.0 and chose to tip Groupmuse $3.0 (the tip appears as an application fee),Argentine Tango Across Continents: Double-Band Live Show with Ben & Winnie (USA) and Asato-Pais (Argentina)
7,13.0,3.00,The attendee paid you $0.0 and chose to tip Groupmuse $3.0 (the tip appears as an application fee),Argentine Tango Across Continents: Double-Band Live Show with Ben & Winnie (USA) and Asato-Pais (Argentina)
8,33.0,3.00,The attendee paid you $0.0 and chose to tip Groupmuse $3.0 (the tip appears as an application fee),Argentine Tango Across Continents: Double-Band Live Show with Ben & Winnie (USA) and Asato-Pais (Argentina)
9,15.0,0.74,Stripe fees of 2.9% + $0.30 = $0.74,Argentine Tango Across Continents: Double-Band Live Show with Ben & Winnie (USA) and Asato-Pais (Argentina)
10,40.0,20.00,The attendee paid you $0.0 and chose to tip Groupmuse $20.0 (the tip appears as an application fee),Argentine Tango Across Continents: Double-Band Live Show with Ben & Winnie (USA) and Asato-Pais (Argentina)
11,23.2,3.00,The attendee paid you $0.0 and chose to tip Groupmuse $3.0 (the tip appears as an application fee),Argentine Tango Across Continents: Double-Band Live Show with Ben & Winnie (USA) and Asato-Pais (Argentina)
12,19.0,3.00,The attendee paid you $0.0 and chose to tip Groupmuse $3.0 (the tip appears as an application fee),Argentine Tango Across Continents: Double-Band Live Show with Ben & Winnie (USA) and Asato-Pais (Argentina)


In [13]:
def adjust_amount(x):
    if x['application_fee (metadata)'].startswith('The attendee paid you'):
        return x['Amount'] - x['Fee']
    else:
        return x['Amount']

event_2_amounts = pd.DataFrame(event_2.apply(adjust_amount, axis = 1))
event_2_amounts.head()

,0
3,20.0
4,50.0
5,50.0
6,50.0
7,10.0


## Write to csv

In [22]:
# event 1
event_1_amounts = event_1[['Amount']].copy()
event_1_amounts['Event'] = 1

# reset the index
event_1_amounts = event_1_amounts.reset_index().drop('index', axis=1)

# save it
# event_1_amounts.to_csv('data/event_1_donations.csv')

In [23]:
# event 2

# reset index
event_2_amounts = event_2_amounts.reset_index().drop('index', axis=1)
event_2_amounts.columns = ['Amount']
event_2_amounts['Event'] = 2

#save it
#event_2_amounts.to_csv('data/event_2_donations.csv')

In [26]:
pd.concat([event_1_amounts, event_2_amounts]).to_csv('data/event_donations.csv')

# Count the number of people who registered for each event

**Event 1**

In [27]:
import requests
from bs4 import BeautifulSoup

In [29]:
def count_guests(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')

    guests = soup.find('div', {'class': 'col-sm-4'})
    guests = guests.findAll('div', {'class' : 'user-content'})
    print(f'There are: {len(guests)} guests signed up.')

    guest_list = []

    for guest in guests:
        print(guest.text)
        guest_list.append(guest.text)


In [31]:
event1_url = 'https://www.groupmuse.com/events/10728-ben-winnie-2hot2tango-online'
event2_url = 'https://www.groupmuse.com/events/11161-argentine-tango-across-continents-double-band-live-show-with-ben-winnie-usa-and-asato-pais-argentina'

In [32]:
# event 1 guests
count_guests(event1_url)

There are: 112 guests signed up.

Benjamin Bogart


Maxfield Wollam-Fisher


Philip Sheegog
Emcee

Joan LaBouff
Joanne Fujii
Lee Ann Elston
Auntie Sarah
Linda Travers
Phil Robison
Yuri Ashuev
Cecil Ho
Zoya Simakhodskaya
Anne and Bill Arnold
Michiko Owaki
Rion Klawinski
Joseph Cheung
Solungga Liu
Joanna Nizynska
Andrew Travers
Arthur Luhur
Laurel and Bill
Deborah Denenfeld
Janice Lee
Karima Cammell
Mary Hepburn
Ray Ihlenburg
Elisa D. Ramos
Werner Duever
Markus Centola
Eric
Mark Danisovszky
Boris Hayete
Patricia and John
Angela Bliss
Tony Dumas
Aaron Travers
Frank Filippi
Jannell Canerday
Sarah Danzman
Andy Alexander
Vernon Sweeney
Music in 2Flavors Música en 2Sabores
Cheryl Sweeney
Leo Cheung
Pablo Zavattieri
Elizabeth Fox
Ed and Sheila
Susan Valley Barton Scheib
Cherry Tsang
Wolfgang
Angelica mclennan
Mark-Hilliard Wilson
Teresa Lawson
Mark J Vanderstelt
Stephanie Grant Knauer
Philip Friedel
Chris nickerson
Rick Usifer
Tara Thurau
Lixin Lin
Misharra Hefler
Kathy Pellegrino and Ben Post

In [33]:
# event 2 guests

count_guests(event2_url)

There are: 128 guests signed up.

James oh


Lily Press
Emcee

Sheldon
Ken Hu
Emily Costa
Marghretta McBean
Brian Gray
Leo Cheung
Aaron Travers
Joan LaBouff
Daniel Inamorato S. Louro
Joanne Fujii
Rion Klawinski
Stephane De Solunne
Chih-yu Chao
Dorothy Read
Carolyn Koh
Linda Travers
Thomas Froman
Leonora Obed
Joel Kilzer
Melinda Grace Fowl
William Turville
Allan Abramson
Joan Glassel
Phil Robison
Elsha Yiu
Alice Loo
Anna Hughes
Leonora Obed
Lourdes Sanchez
Frank Filippi
Auntie Sarah
Tom Kenney
Johan van Zyl
Joseph Cheung
Leonidas Ramirez Riveros
Vin de Silva
Anvi Hoàng
Syed Shamsuddoha
Pablo Zavattieri
Zoya Simakhodskaya
Valerie A. Higgs
Bryn
Cheryl Sweeney
Nathan Koch
Maria Centola
Mark J Vanderstelt
張慧雄
Adena Staben
Michiko Owaki
Peilin Chiu
Sandra Freund
Ruth rozelaar
Yuri Ashuev
Joe Grohens
Rick Usifer
Jose Halfon
Lya Elcagu
Chris B-Underground
Andy Alexander
Amin Jobs
Concepcion Tadeo
Koa Hons
Elaine Gerbert
Gregory Levinson
Mark-Hilliard Wilson
Katrina Lundin Davis
Paola Bordon
La